In [3]:
import os
import re
import nibabel as nib
import numpy as np
import torch
from nipype.interfaces import fsl
from tqdm import tqdm
import glob

def find_func_file(patient_dir):
    for root, dirs, files in os.walk(patient_dir):
        for file in files:
            if file.endswith('_bold.nii.gz'):
                return os.path.join(root, file)
    return None

def run_bet(in_file, out_file, mask_file):
    print(f"Running BET on {in_file}")
    bet = fsl.BET(in_file=in_file, out_file=out_file, mask=True, frac=0.3)
    result = bet.run()
    
    possible_mask_files = glob.glob(out_file.replace('.nii.gz', '') + '*mask.nii.gz')
    
    if possible_mask_files:
        os.rename(possible_mask_files[0], mask_file)
    else:
        bet_img = nib.load(out_file)
        bet_data = bet_img.get_fdata()
        mask_data = (bet_data > 0).astype(np.int16)
        mask_img = nib.Nifti1Image(mask_data, bet_img.affine, bet_img.header)
        nib.save(mask_img, mask_file)
    
    return result

def run_flirt(in_file, ref_file, out_file, out_matrix_file, dof=12):
    print(f"Running FLIRT on {in_file}")
    flirt = fsl.FLIRT()
    flirt.inputs.in_file = in_file
    flirt.inputs.reference = ref_file
    flirt.inputs.out_file = out_file
    flirt.inputs.out_matrix_file = out_matrix_file
    flirt.inputs.dof = dof
    flirt.inputs.cost = 'corratio'
    return flirt.run()

def apply_mask(in_file, mask_file, out_file):
    print(f"Applying mask to {in_file}")
    apply_mask = fsl.ApplyMask()
    apply_mask.inputs.in_file = in_file
    apply_mask.inputs.mask_file = mask_file
    apply_mask.inputs.out_file = out_file
    return apply_mask.run()

def preprocess_fmri_3d_pointing(func_file, standard_file, output_dir, subject_id, session_id):
    print(f"Preprocessing fMRI data for subject {subject_id}, session {session_id}")
    os.makedirs(output_dir, exist_ok=True)

    func_img = nib.load(func_file)
    func_data = func_img.get_fdata()
    num_volumes = func_data.shape[3]

    subject_output_dir = os.path.join(output_dir, 'img', f"sub-{subject_id}_ses-{session_id}")
    os.makedirs(subject_output_dir, exist_ok=True)

    # Extract first volume and run BET
    first_volume = func_data[:,:,:,0]
    first_volume_img = nib.Nifti1Image(first_volume, func_img.affine)
    first_volume_file = os.path.join(output_dir, "first_volume.nii.gz")
    nib.save(first_volume_img, first_volume_file)

    bet_output = os.path.join(output_dir, "bet_first_volume.nii.gz")
    mask_file = os.path.join(output_dir, "brain_mask.nii.gz")
    run_bet(first_volume_file, bet_output, mask_file)

    for i in tqdm(range(num_volumes), desc=f"Processing volumes for subject {subject_id}"):
        volume = func_data[:,:,:,i]
        volume_img = nib.Nifti1Image(volume, func_img.affine)
        volume_file = os.path.join(output_dir, f"volume_{i}.nii.gz")
        nib.save(volume_img, volume_file)

        # Apply mask
        masked_volume = os.path.join(output_dir, f"masked_volume_{i}.nii.gz")
        apply_mask(volume_file, mask_file, masked_volume)

        # Register to MNI space
        mni_output = os.path.join(output_dir, f"mni_volume_{i}.nii.gz")
        mni_matrix = os.path.join(output_dir, f"mni_matrix_{i}.mat")
        run_flirt(masked_volume, standard_file, mni_output, mni_matrix, dof=12)

        mni_volume = nib.load(mni_output).get_fdata()
        mni_volume_norm = (mni_volume - np.mean(mni_volume)) / np.std(mni_volume)

        # Save as .pt file
        torch.save(torch.from_numpy(mni_volume_norm).float(), 
                   os.path.join(subject_output_dir, f"frame_{i:04d}.pt"))

        for file in [volume_file, masked_volume, mni_output, mni_matrix]:
            if os.path.exists(file):
                os.remove(file)

    # Clean up
    os.remove(first_volume_file)
    os.remove(bet_output)
    os.remove(mask_file)

def process_subject(patient_dir, output_dir, standard_file):
    subject_id = os.path.basename(patient_dir).split('_')[0].replace('OAS', '')
    session_id = os.path.basename(patient_dir).split('_')[2].replace('d', '')
    
    func_file = find_func_file(patient_dir)
    
    if func_file:
        print(f"Found functional file: {func_file}")
        preprocess_fmri_3d_pointing(func_file, standard_file, output_dir, subject_id, session_id)
        return subject_id, session_id
    else:
        print(f"No matching functional file found for subject {subject_id}, session {session_id}")
        return None

def process_all_subjects(base_dir, output_dir, standard_file):
    subjects_processed = []

    for patient_folder in os.listdir(base_dir):
        patient_dir = os.path.join(base_dir, patient_folder)
        if not os.path.isdir(patient_dir):
            continue
        
        result = process_subject(patient_dir, output_dir, standard_file)
        if result:
            subjects_processed.append(result)

    return subjects_processed

# 메인 실행 셀
base_dir = "/home/minyoungxi/MINYOUNGXI/XFL/Data/OASISSamples7.1"
output_dir = "/home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs"
standard_file = os.path.join(os.environ['FSLDIR'], 'data', 'standard', 'MNI152_T1_1mm_brain.nii.gz')

print("Starting preprocessing...")
subjects_processed = process_all_subjects(base_dir, output_dir, standard_file)
print(f"Preprocessing completed. {len(subjects_processed)} subjects processed.")

print("Preprocessing and data structuring completed.")

Starting preprocessing...
Found functional file: /home/minyoungxi/MINYOUNGXI/XFL/Data/OASISSamples7.1/OAS30033_MR_d0133/func1/NIFTI/sub-OAS30033_ses-d0133_task-rest_run-01_bold.nii.gz
Preprocessing fMRI data for subject 30033, session 0133
Running BET on /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/first_volume.nii.gz


Processing volumes for subject 30033:   0%|          | 0/164 [00:00<?, ?it/s]

Applying mask to /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/volume_0.nii.gz
Running FLIRT on /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/masked_volume_0.nii.gz


Processing volumes for subject 30033:   1%|          | 1/164 [00:04<13:26,  4.94s/it]

Applying mask to /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/volume_1.nii.gz
Running FLIRT on /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/masked_volume_1.nii.gz


...

Applying mask to /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/volume_162.nii.gz
Running FLIRT on /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/masked_volume_162.nii.gz


Processing volumes for subject 30061:  99%|█████████▉| 163/164 [13:59<00:05,  5.20s/it]

Applying mask to /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/volume_163.nii.gz
Running FLIRT on /home/minyoungxi/MINYOUNGXI/XFL/Data/OASIS_MNI_to_TRs/masked_volume_163.nii.gz


Processing volumes for subject 30061: 100%|██████████| 164/164 [14:04<00:00,  5.15s/it]

Preprocessing completed. 10 subjects processed.
Preprocessing and data structuring completed.
